In [1]:
import csv
import math
import time
import random
import networkx as nx
import numpy as np
from copy import deepcopy
import pandas as pd
from rdkit import Chem
from tqdm import tqdm

import os
import pandas as pd

import pickle

import pandas as pd
from rdkit.Chem import rdmolfiles
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize

import torch
import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

# rom torch_scatter import scatter
# from torch_geometric.data import Data, Dataset, DataLoader

import rdkit
from rdkit import Chem
from rdkit.Chem.rdchem import HybridizationType
from rdkit.Chem.rdchem import BondType as BT
from rdkit.Chem import AllChem

from rdkit.Chem import rdMolTransforms
from rdkit.Chem.Draw import rdMolDraw2D, rdDepictor, IPythonConsole

rdDepictor.SetPreferCoordGen(True)
from IPython.display import Image

tqdm.pandas()
from rdkit import RDLogger

RDLogger.DisableLog('rdApp.*')  

/home/nick1899/anaconda3/envs/mol/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_ecfp(smiles, radius):
    mol = AllChem.MolFromSmiles(smiles, sanitize=False)
    mol = rdMolStandardize.Normalize(mol)
    try:
        Chem.SanitizeMol(mol)
    except Chem.AtomValenceException:
        return None
    radii = list(range(int(radius) + 1))
    info = {}
    _ = AllChem.GetMorganFingerprint(mol, radius, bitInfo=info)  # info: dictionary identifier, atom_idx, radius

    mol_atoms = [a.GetIdx() for a in mol.GetAtoms()]
    dict_atoms = {x: {r: None for r in radii} for x in mol_atoms}

    for element in info:
        for atom_idx, radius_at in info[element]:
            dict_atoms[atom_idx][radius_at] = element  # {atom number: {fp radius: identifier}}

        # iterate over all atoms and radii
    identifier_sentences = []

    for r in radii:  # iterate over radii to get one sentence per radius
        identifiers = []
        for atom in dict_atoms:  # iterate over atoms
            # get one sentence per radius
            identifiers.append(dict_atoms[atom][r])
        identifier_sentences.append(list(map(str, [x for x in identifiers if x])))
    return list(identifier_sentences)

In [21]:
import pickle 

'''with open('filename.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)'''
with open('filename_uncut.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [22]:
data.head()

,index,Unnamed: 0,smiles,ecfp5,ecfp1,ecfp2,ecfp3,ecfp4,ecfp1_cut,ecfp2_cut,ecfp3_cut,ecfp4_cut,ecfp5_cut
0,2859816,2859816,N#CC(=CNc1ccc(Nc2ccccc2)cc1)C(=O)N1CCCCC1,"[[847433064, 2245900962, 2246699815, 224670379...","[[847433064, 2245900962, 2246699815, 224670379...","[[847433064, 2245900962, 2246699815, 224670379...","[[847433064, 2245900962, 2246699815, 224670379...","[[847433064, 2245900962, 2246699815, 224670379...","[2551483158, 781447971, 554582195, 3880858139,...","[3194233911, 4235880976, 1567584741, 367703104...","[2280706294, 541892508, 2866997053, 240765344,...","[410772916, 876894891, 88381873, 667131381, 23...","[600531453, 2511373375, 3759280899, 1474317629..."
1,7256086,7256086,O=C1c2ccccc2NC(c2ccc(SC(F)(F)F)cc2)N1CC1CCCO1,"[[864942730, 3217380708, 3217380708, 321869396...","[[864942730, 3217380708, 3217380708, 321869396...","[[864942730, 3217380708, 3217380708, 321869396...","[[864942730, 3217380708, 3217380708, 321869396...","[[864942730, 3217380708, 3217380708, 321869396...","[10565946, 1871221615, 3982076256, 951226070, ...","[1513450307, 3476417958, 1323467736, 399990699...","[4057506034, 1165060899, 900277316, 3488480553...","[4264674485, 3128146264, 1544352305, 264116875...","[67173608, 2254477329, 3855819969, 3157912345,..."
2,5926236,5926236,CSCCC([NH3+])C(=O)NC(C)c1ccccc1C,"[[2246728737, 1026928756, 2245384272, 22453842...","[[2246728737, 1026928756, 2245384272, 22453842...","[[2246728737, 1026928756, 2245384272, 22453842...","[[2246728737, 1026928756, 2245384272, 22453842...","[[2246728737, 1026928756, 2245384272, 22453842...","[2592252298, 229197718, 4120224926, 1583799011...","[3673190411, 2051816982, 824615024, 2484722359...","[1171484618, 2534182396, 473601359, 1274470318...","[1896159405, 3508375579, 1637613376, 413671164...","[1464203311, 1562612457, 2973709734, 3255664645]"
3,6522590,6522590,CCCOc1ccc(C(=O)Nc2nnc(S(=O)(=O)Cc3ccccc3F)s2)cc1,"[[2246728737, 2245384272, 2245384272, 86467448...","[[2246728737, 2245384272, 2245384272, 86467448...","[[2246728737, 2245384272, 2245384272, 86467448...","[[2246728737, 2245384272, 2245384272, 86467448...","[[2246728737, 2245384272, 2245384272, 86467448...","[3542456614, 1173125914, 4008337421, 211547690...","[2636383078, 3635585100, 717151242, 910450790,...","[1047480442, 1054157635, 691974566, 3202522643...","[1997538764, 1774883767, 1613582335, 403948141...","[2376722083, 210371865, 2657191478, 2749592338..."
4,3587030,3587030,COc1ccc(Br)c(C(=O)Nc2cc(NC(C)=O)ccc2OC)c1,"[[2246728737, 864674487, 3217380708, 321869396...","[[2246728737, 864674487, 3217380708, 321869396...","[[2246728737, 864674487, 3217380708, 321869396...","[[2246728737, 864674487, 3217380708, 321869396...","[[2246728737, 864674487, 3217380708, 321869396...","[3975275337, 2076190208, 3162837314, 951226070...","[1352399629, 1840994228, 2944555726, 181468282...","[2287205162, 3805467668, 1207405495, 346140264...","[1970961427, 2085320272, 2012833611, 273556979...","[4231106260, 2494039993, 3934723367, 295828982..."


In [5]:
len(data['ecfp5'])

10000

In [6]:
'''radius = 'ecfp0'
folder = 'ecfps_full'
filename = folder + '/' + radius'''

"radius = 'ecfp0'\nfolder = 'ecfps_full'\nfilename = folder + '/' + radius"

In [7]:
# just ecfp0's in one file splitted by \n
'''with open('../' + filename + '.txt', 'r') as fp: 
    text = fp.read().split('\n')'''

"with open('../' + filename + '.txt', 'r') as fp: \n    text = fp.read().split('\n')"

In [23]:
def transform_column(row):
    # Flatten the 2D array and join with a space
    row = [row]
    flat_list = [str(item) for sublist in row for item in sublist]  # Flatten
    return ' '.join(flat_list)  # Join with space

# Apply the transformation and join with dots
#df['ecfp5_transformed'] = df['ecfp5'].apply(transform_column)
x = data['ecfp2_cut'].iloc[:100000].apply(transform_column)
result = ' . '.join(x)

In [ ]:
from word2vec import Word2VecWrapper

vocab_size = 1000
text = result
wrapper = Word2VecWrapper(vocab_size, text)

In [ ]:
wrapper.preprocess()

wrapper.remove_low_frequency_tokens(threshold=5, constant_value=-1)

tokens_ids = wrapper.get_tokens_ids()

37202


In [ ]:
tokens_ids

In [ ]:
wrapper.train()

32.402416706085205


In [ ]:
embedding = wrapper.get_embedding(32781)
print(embedding)

[ 0.0317356   0.17633578 -0.30952597 -0.0542495   0.19110513 -0.38696617
  0.16651958  0.44681567 -0.23935911 -0.07243555 -0.12634794 -0.13582334
  0.09584567  0.03816165  0.01442356  0.19227102 -0.00149756 -0.1963766
  0.14633353 -0.48048115  0.13660276  0.00487293  0.51033753 -0.27315235
 -0.18558194  0.07316709 -0.2742482  -0.07636286 -0.1349674   0.0892311
 -0.01866038 -0.12843658  0.33709183 -0.08459272 -0.0122526   0.21886465
 -0.05638981 -0.08785342 -0.26152658 -0.10343356  0.1757676  -0.13320576
  0.00301688  0.04394046 -0.09462924 -0.17342493 -0.10549347 -0.21686797
  0.19695155  0.08532552  0.01386155 -0.20083714  0.01878696  0.04710573
 -0.02377879 -0.03654465 -0.03590859  0.00247921 -0.3462912   0.12331245
 -0.07666664  0.0663688   0.01805799  0.03846407 -0.03999834  0.3992568
  0.06402408  0.0014625  -0.16847911  0.06082245 -0.18305044  0.07976504
  0.13552655 -0.16261846  0.3421547   0.09809462  0.30726382  0.10982297
 -0.08539578  0.21957688 -0.2374775   0.10748871 -0.22

In [45]:
def get_folded_counts(molecules: List[List[int]], max_factor: int) -> Dict[Any, int]: # expects max_factor to be <= 4 as complexity has exp-growth factor
    count = defaultdict(int)
    for molecule in molecules:
        subsets = combinations(sorted(molecule), max_factor) # "If the input iterable is sorted, the output tuples will be produced in sorted order." (c) docs
        for subset in list(subsets):
            if -1 in subset: # skip junk (?)
                continue
            item = subset[:] if max_factor > 1 else subset[0]
            count[item] += 1
    return count
 
 
print(dict(get_folded_counts([[1, 2, 3], [3, 4, 5], [5, 4, 3]], 1)))
print(dict(get_folded_counts([[1, 2, 3], [3, 4, 5], [5, 4, 3]], 2)))

{1: 1, 2: 1, 3: 3, 4: 2, 5: 2}
{(1, 2): 1, (1, 3): 1, (2, 3): 1, (3, 4): 2, (3, 5): 2, (4, 5): 2}


In [46]:
for molecule in token_ids:
    molecule.sort() # for determinism
 
base_counts = get_folded_counts(token_ids, 1)
groups_by_2_counts =  get_folded_counts(token_ids, 2)
# print(base_counts)
# print(groups_by_2_counts)
 


In [47]:
hits = 0
to_drop = set()
to_unify = set()
for pair, pair_counts in groups_by_2_counts.items():
    a, b = pair
    if a == -1 or b == -1: continue
 
    a_count = base_counts[a]
    b_count = base_counts[b]
    if pair_counts >= max(a_count, b_count): # a_count + b_count - pair_counts # TODO: probably, change statement. May be add some threshold
        to_drop.add(a)
        to_drop.add(b)
        to_unify.add((a, b))
        # print(a,b, a_count, b_count, pair_counts)
        hits += 1
print("Replace count: ", hits)
 
 
pairs_replace_mapping = {}
for molecule in token_ids: # Put new tokens for popular groups
    new_molecule = molecule
    pairs_list = list(combinations(new_molecule, 2))
    sorted_pairs = sorted(pairs_list, key=lambda x: groups_by_2_counts.get(x, 0), reverse=True)
    #print("old", molecule)
    for group in sorted_pairs: # make combinations(new_molecule, 2) sorted by frequancy
        if group in to_unify:
            # print(group, " !")
            if group not in list(pairs_replace_mapping.keys()):
                new_group_id = random.randint(list(updated_inverse_vocab.items())[-1:][0][1], vocab_size) # TODO
                pairs_replace_mapping[group] = new_group_id
            else:
                new_group_id = pairs_replace_mapping[group]
            if group[1] in new_molecule and group[0] in new_molecule:
                if group[1] == group[0]:
                    if new_molecule.count(group[1]) >= 2:
                        new_molecule.remove(group[0])
                        new_molecule.append(new_group_id)
                        new_molecule.remove(group[1])
                        #print(group, new_group_id)
                else:
                    new_molecule.remove(group[0])
                    new_molecule.append(new_group_id)
                    new_molecule.remove(group[1])
                    #print(group, new_group_id)
    molecule = new_molecule
    #print("new", molecule)
    #print("")
    
 
 
 
for molecule in token_ids: # Evict old atoms which got into groups
    molecule = list(map(lambda atom: -1 if atom in to_drop else atom, molecule))

Replace count:  28
